In [1]:
from transformers import AutoTokenizer
fb_model = "facebook/blenderbot-400M-distill"  #
tokenizer = AutoTokenizer.from_pretrained(fb_model)

tokenizer_config.json:   0%|          | 0.00/1.15k [00:00<?, ?B/s]

config.json:   0%|          | 0.00/1.57k [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/127k [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/62.9k [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/16.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/772 [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/310k [00:00<?, ?B/s]

In [3]:
chat = [
    {"role": "user","content": "Hello, How are you"},
    {"role": "assistant", "content": "I am doing good. How can I help you"},
    {"role": "user", "content": "How can I show off chat template"}
]

In [4]:
tokenizer.apply_chat_template(chat, tokenize=False)


No chat template is defined for this tokenizer - using the default template for the BlenderbotTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



' Hello, How are you  I am doing good. How can I help you   How can I show off chat template</s>'

In [5]:
mistral_model = "mistralai/Mistral-7B-Instruct-v0.1"  # 750 MB model
tokenizer = AutoTokenizer.from_pretrained(mistral_model)

tokenizer.apply_chat_template(chat, tokenize=False)

# he tokenizer has added the control tokens [INST] and [/INST] to indicate the 
# start and end of user messages (but not assistant messages!)

tokenizer_config.json:   0%|          | 0.00/1.47k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/72.0 [00:00<?, ?B/s]

'<s>[INST] Hello, How are you [/INST]I am doing good. How can I help you</s> [INST] How can I show off chat template [/INST]'

When using chat templates as input for model generation, it’s also a good idea to use add_generation_prompt=True to add a generation prompt.

In [ ]:
from transformers import AutoModelForCausalLM

checkpoint = "HuggingFaceH4/zephyr-7b-beta"  # 15GB model

model = AutoModelForCausalLM.from_pretrained(checkpoint)  # You may want to use bfloat16 and/or move to GPU here
tokenizer = AutoTokenizer.from_pretrained(checkpoint)
messages = [
    {"role": "system","content": "You are a friendly chatbot who always responds in the style of a pirate",},
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
 ]

tokenized_chat = tokenizer.apply_chat_template(messages,
                                               tokenize=True,
                                               add_generation_prompt=True, return_tensors="pt")

print(tokenizer.decode(tokenized_chat[0]))

# if using fb_model, the system followed by user roles threw error

config.json:   0%|          | 0.00/638 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00008.safetensors:   0%|          | 0.00/1.89G [00:00<?, ?B/s]

model-00002-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00003-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

...

model-00005-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00006-of-00008.safetensors:   0%|          | 0.00/1.95G [00:00<?, ?B/s]

model-00007-of-00008.safetensors:   0%|          | 0.00/1.98G [00:00<?, ?B/s]

model-00008-of-00008.safetensors:   0%|          | 0.00/816M [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/8 [00:00<?, ?it/s]

In [ ]:
outputs = model.generate(tokenized_chat, max_new_tokens=128) 
print(tokenizer.decode(outputs[0]))

In [ ]:
from transformers import pipeline

pipe = pipeline("conversational", "HuggingFaceH4/zephyr-7b-beta")
# The pipeline type called conversational, used for chat modeling.

messages = [
    {
        "role": "system",
        "content": "You are a friendly chatbot who always responds in the style of a pirate",
    },
    {"role": "user", "content": "How many helicopters can a human eat in one sitting?"},
]
print(pipe(messages))

# ConversationalPipeline will take care of all the details of tokenization and calling 
# apply_chat_template for you - once the model has a chat template, 

In [ ]:
messages = [
    {"role": "user", "content": "Hi there!"},
    {"role": "assistant", "content": "Nice to meet you!"},
    {"role": "user", "content": "Can I ask a question?"}
]

In [ ]:
tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=False)

"""<|im_start|>user
Hi there!<|im_end|>
<|im_start|>assistant
Nice to meet you!<|im_end|>
<|im_start|>user
Can I ask a question?<|im_end|>
"""

s. Some models, like **BlenderBot** and **LLaMA**, don’t have any special tokens before bot responses. 

In [ ]:
tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt=True)

"""<|im_start|>user
Hi there!<|im_end|>
<|im_start|>assistant
Nice to meet you!<|im_end|>
<|im_start|>user
Can I ask a question?<|im_end|>
<|im_start|>assistant
"""
# note the <|im_start|>assistant at the end. Will prompt for assistant to respond

In [2]:
from datasets import Dataset
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("HuggingFaceH4/zephyr-7b-beta")

chat1 = [
    {"role": "user", "content": "Which is bigger, the moon or the sun?"},
    {"role": "assistant", "content": "The sun."}
]
chat2 = [
    {"role": "user", "content": "Which is bigger, a virus or a bacterium?"},
    {"role": "assistant", "content": "A bacterium."}
]

dataset = Dataset.from_dict({"chat": [chat1, chat2]})
dataset = dataset.map(lambda x: {"formatted_chat": tokenizer.apply_chat_template(x["chat"],
                                                                                 tokenize=False,
                                                                                 add_generation_prompt=False)
                                }
                     )
print(dataset['formatted_chat'][0])

tokenizer_config.json:   0%|          | 0.00/1.43k [00:00<?, ?B/s]

tokenizer.model:   0%|          | 0.00/493k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.80M [00:00<?, ?B/s]

added_tokens.json:   0%|          | 0.00/42.0 [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/168 [00:00<?, ?B/s]

  0%|          | 0/2 [00:00<?, ?ex/s]

<|user|>
Which is bigger, the moon or the sun?</s>
<|assistant|>
The sun.</s>



In [3]:
from transformers import AutoTokenizer
tokenizer = AutoTokenizer.from_pretrained("facebook/blenderbot-400M-distill")

In [4]:
from rich import print

print(tokenizer.default_chat_template)


No chat template is defined for this tokenizer - using the default template for the BlenderbotTokenizerFast class. If the default is not appropriate for your model, please set `tokenizer.chat_template` to an appropriate template. See https://huggingface.co/docs/transformers/main/chat_templating for more information.



{% for message in messages %}{% if message['role'] == 'user' %}{{ ' ' }}{% endif %}{{ message['content'] }}{% if 
not loop.last %}{{ '  ' }}{% endif %}{% endfor %}{{ eos_token }}

{% for message in messages %}
    
    {% if message['role'] == 'user' %}
    
        {{ ' ' }}
    
    {% endif %}
    
    {{ message['content'] }}
    
    {% if not loop.last %}
    
        {{ '  ' }}
    
    {% endif %}

{% endfor %}

{{ eos_token }}

In [ ]:
for idx, message in enumerate(messages):
    if message['role'] == 'user':
        print(' ')
    print(message['content'])
    if not idx == len(messages) - 1:  # Check for the last message in the conversation
        print('  ')
print(eos_token)

"""
Effectively, the template does three things:

For each message, if the message is a user message, add a blank space before it,
otherwise print nothing.

Add the message content

If the message is not the last message, add two spaces after it. 
After the final message, print the EOS token.
"""

template = {% for message in messages %}
    
    {% if message['role'] == 'user' %}
    
        {{ bos_token + '[INST] ' + message['content'] + ' [/INST]' }}
    
    {% elif message['role'] == 'system' %}
    
        {{ '<<SYS>>\\n' + message['content'] + '\\n<</SYS>>\\n\\n' }}
    
    {% elif message['role'] == 'assistant' %}
        
        {{ ' '  + message['content'] + ' ' + eos_token }}
    
    {% endif %}

{% endfor %}

###  it adds specific tokens based on the “role” of each message, which represents who sent it. User, assistant and system messages are clearly distinguishable to the model because of the tokens they’re wrapped in. 

You may find it easier to start with an existing template from another model and simply edit it for your needs! For example, we could take the LLaMA template above and add ”[ASST]” and ”[/ASST]” to assistant messages:

In [ ]:
template = tokenizer.chat_template
template = template.replace("SYS", "SYSTEM")  # Change the system token
tokenizer.chat_template = template  # Set the new template
tokenizer.push_to_hub("model_name")  # Upload your new template to the Hub!

You can find out what the **default template** for your tokenizer is by checking the tokenizer.default_chat_template attribute.

### How to choose the template

- ensure that the template exactly matches the message formatting that the model saw during training

- you will probably get the best performance if you keep the chat tokens constant

- the ChatML format, and this is a good, flexible choice for many use-cases

{% for message in messages %}
    
    {{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}

{% endfor %}

In [5]:
# The one-liner also includes handy support for generation prompts, but note that
# it doesn’t add BOS or EOS tokens! If your model expects those, 
# they won’t be added automatically by apply_chat_template - in other words, 
# the text will be tokenized with add_special_tokens=False.

tokenizer.chat_template = """{% if not add_generation_prompt is defined %}{% set add_generation_prompt = false %}{% endif %}{% for message in messages %}{{'<|im_start|>' + message['role'] + '\n' + message['content'] + '<|im_end|>' + '\n'}}{% endfor %}{% if add_generation_prompt %}{{ '<|im_start|>assistant\n' }}{% endif %}"""

If you’re unfamiliar with Jinja, we generally find that the easiest way to write a chat template is to first write a short Python script that formats messages the way you want, and then convert that script into a template.

> for loop:

{% for message in messages %}

{{ message['content'] }}

{% endfor %}

> {{ expression block }} is printed

> If Block:

{% if message['role'] == 'user' %}

{{ message['content'] }}

{% endif %}

> Jinja requires you to explicitly end them with {% endfor %} and {% endif %}.

> Inside your template, you will have access to the list of messages, but you can also access several other special variables. These include special tokens like bos_token and eos_token, as well as the add_generation_prompt variable that we discussed above

> , we’ve tried to get Jinja to ignore whitespace outside of {{ expressions }}. However, be aware that Jinja is a general-purpose templating engine, and it may treat whitespace between blocks on the same line as significant and print it to the output